# Closures

In [30]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [31]:
fn  = outer()
fn.__code__.co_freevars

('x',)

As we can see, `x` is a free variable in the closure.

In [32]:
fn.__closure__

(<cell at 0x10708eb90: str object at 0x1051af5b0>,)

Here we see that the free variable x is actually a reference to a cell object that is itself a reference to a string object.

Let's see what the memory address of `x` is in the outer function and the inner function. To be sure string interning does not play a role, I am going to use an object that we know Python will not automatically intern, like a list.

In [68]:
def outer():
    x = [1,2,3]
    print("Outer:",hex(id(x)))
    def inner():
        y=x
        print("Inner:",hex(id(y)))
        print(x)
    return inner

In [69]:
fn = outer()

Outer: 0x1070b8680


In [70]:
fn.__closure__

(<cell at 0x1070e3640: list object at 0x1070b8680>,)

In [71]:
fn()

Inner: 0x1070b8680
[1, 2, 3]


### Meddefying free variables
We know we can modify nonlocal variables by using the `nonlocal` keyword. So the following will work:

In [72]:
def Counter():
    count = 0
    def inc():
        nonlocal count
        count +=1
        return count
    return inc

In [73]:
fn = Counter()

In [76]:
fn()

1

In [77]:
fn.__code__.co_freevars

('count',)

In [75]:
fn.__closure__

(<cell at 0x1070e3880: int object at 0x10438ed50>,)

In [41]:
hex(id(0))

'0x10438ed50'

Here free variable is pointinf to int object zero : 0,
because both zeros are pointing to same memory location (interning)

In [42]:
fn()

1

In [43]:
fn.__closure__

(<cell at 0x10708e320: int object at 0x10438ed70>,)

In [44]:
hex(id(1))
# Same address as above because of interning

'0x10438ed70'

##### Shared Extended Scopes
As we saw in the lecture, we can set up nonlocal variables in different inner functions that reference the same outer scope variable, i.e. we have a free variable that is shared between two closures. This works because both non local variables and the outer local variable all point back to the same cell object.

Now memory address of the free variable is changed,  i.e it has modefied count's value from zero to one

In [45]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count +=1
        return count
    
    def inc2():
        nonlocal count 
        count +=1
        return count
        
    return inc1,inc2

In [46]:
fn1,fn2 = outer()

In [47]:
fn1.__closure__,fn2.__closure__

((<cell at 0x10708d9f0: int object at 0x10438ed50>,),
 (<cell at 0x10708d9f0: int object at 0x10438ed50>,))

In [48]:
fn1() #modefying count using first block (inc1)

1

In [49]:
fn1.__closure__,fn2.__closure__

((<cell at 0x10708d9f0: int object at 0x10438ed70>,),
 (<cell at 0x10708d9f0: int object at 0x10438ed70>,))

In [50]:
fn2() #modefying count using first block (inc2)

2

In [51]:
fn1.__closure__,fn2.__closure__

((<cell at 0x10708d9f0: int object at 0x10438ed90>,),
 (<cell at 0x10708d9f0: int object at 0x10438ed90>,))

Here we can see that cell address remains the same but object address changes based on the value of count

### Multiple Instances of Closures
Recall that **every** time a function is called, a **new** local scope is created.

In [78]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [79]:
func()

31557.499567333 4411034576


In [80]:
func()

31573.697951625 4411034928


#### Example
The same thing happens with closures, they have their own extended scope every time the closure is created:

In [52]:
def pow(n):
    def inner(x):
        return x**n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that contains `inner` and the free variable `n`

In [58]:
sq = pow(2)
sq.__closure__

(<cell at 0x10708e9e0: int object at 0x10438ed90>,)

In [54]:
sq(2)

4

In [59]:
cube = pow(3)
cube.__closure__

(<cell at 0x10708e620: int object at 0x10438edb0>,)

In [81]:
cube(3)

27

We can see a the cell used for the free variable in both cases is **different**:

Here every call gets a new scope, therefore it created diffrent closures each time

In fact, these functions (`square` and `cube`) are **not** the same functions, even though they were "created" from the same `power` function:

In [82]:
id(sq), id(cube)

(4413087776, 4413087936)

### Beware!
Remember when I said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called?

This can create very subtle bugs in your program.

Consider the following example where we want to create some functions that can add 1, 2, 3, 4 and to whatever is passed to them.

In [83]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [84]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [85]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

But suppose we want to be little fancier then we can do like so.

In [87]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [88]:
adders = create_adders()

Now technically we have 4 functions in the `adders` list:

In [89]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]